In [7]:
import arff
with open('data.arff', 'r') as f:
    data = arff.load(f)

  Preparing metadata (setup.py) ... done
  Created wheel for liac-arff: filename=liac_arff-2.5.0-py3-none-any.whl size=11717 sha256=9275c351c3ecaaef4c97d843da450cf4efa32f583fe23f8672f3e85acadc046c
  Stored in directory: /Users/richard/Library/Caches/pip/wheels/df/2d/e9/05702e5ac2ac33a006f12c36e9b8f32f59f0050a616978ec31
Successfully built liac-arff

[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
